In [5]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import os
%cd /cluster2/home/futing/Project/HiCQTL/merged/CRC53g/covariate
prefix='CRCgeno10'

/cluster2/home/futing/Project/HiCQTL/merged/CRC53g/covariate


#### 合并 genoPC 和 cooler info

In [6]:

genoPC=pd.read_csv(f'{prefix}.eigenvec',sep='\t') #,header=None

# genoPC=genoPC.drop(columns=[0]) # 去掉第一列
genoPC.columns=['cell']+["PC" + str(i) for i in range(1,11)] # 设置列名
genoPC.head()

,cell,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10
0,09-376,-0.012284,0.036774,-0.010078,-0.006224,0.003777,-0.001598,0.005694,0.005845,0.019016,0.018068
1,11-1026,-0.014879,0.034419,-0.011229,0.028991,0.006685,0.000259,0.009118,-0.007509,0.014384,0.013685
2,11-1074,-0.011783,0.044870,-0.020293,-0.002919,-0.061093,0.242603,-0.622112,0.057411,0.024371,-0.113733
3,11-1074_Normal,-0.011526,0.044422,-0.020083,-0.007265,-0.058645,0.236657,-0.601014,0.055795,0.024659,-0.109545
4,11-51,-0.015352,0.041232,-0.019142,0.046098,0.025357,-0.084334,0.178513,-0.019549,0.379304,-0.431482


In [8]:
coolinfo=pd.read_csv('/cluster2/home/futing/Project/panCancer/Analysis/QC/nContacts/hicInfo_Aug04.txt',sep='\t')
coolinfo['element']=coolinfo['element'].str.replace(',','')

In [9]:
covariate=pd.merge(coolinfo[['cell','element']],genoPC,on='cell',how='right')
covariate=covariate.set_index('cell')
covariate['tumor'] = [0 if x.endswith('_Normal') else 1 for x in covariate.index]
# covariate = covariate.T
covariate.head(8)

,element,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,tumor
cell,,,,,,,,,,,,
09-376,105179165,-0.012284,0.036774,-0.010078,-0.006224,0.003777,-0.001598,0.005694,0.005845,0.019016,0.018068,1
11-1026,102757999,-0.014879,0.034419,-0.011229,0.028991,0.006685,0.000259,0.009118,-0.007509,0.014384,0.013685,1
11-1074,95497276,-0.011783,0.044870,-0.020293,-0.002919,-0.061093,0.242603,-0.622112,0.057411,0.024371,-0.113733,1
11-1074_Normal,90275817,-0.011526,0.044422,-0.020083,-0.007265,-0.058645,0.236657,-0.601014,0.055795,0.024659,-0.109545,0
11-51,61174254,-0.015352,0.041232,-0.019142,0.046098,0.025357,-0.084334,0.178513,-0.019549,0.379304,-0.431482,1
11-51_Normal,113095102,-0.013451,0.041925,-0.020580,0.040958,0.021966,-0.079938,0.176040,-0.019223,0.373424,-0.429827,0
11-52,94723091,-0.015114,0.044304,-0.017271,0.297622,0.567393,-0.038378,0.025191,0.197111,-0.355299,-0.064153,1
11-52_Normal,87713161,-0.016796,0.040747,-0.011226,0.231227,0.439861,-0.026159,0.008160,0.149510,-0.248623,-0.046648,0


### 合并 phenoPC genoPC 和 cooler info

In [18]:
phenoprefix='CRC53hard'
filelist=pd.read_csv(f'../genotype/{phenoprefix}/{phenoprefix}.psam',sep='\t') #genotype的顺序



In [19]:
expr=pd.read_csv('/cluster2/home/futing/Project/HiCQTL/merged/CRC53g/phenotype/CRC53g_0901.bed',sep='\t')
fixed = expr.iloc[:, :4]
reordered = expr.loc[:, filelist['#IID']]
expr_new = pd.concat([fixed, reordered], axis=1)

expr_new.head()

/tmp/ipykernel_35784/1284482933.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  expr=pd.read_csv('/cluster2/home/futing/Project/HiCQTL/merged/CRC53g/phenotype/CRC53g_0901.bed',sep='\t')


,chr,start,end,gene,09-376,11-1026,11-1074,11-1074_Normal,11-51,11-51_Normal,...,14-91,15-1382,15-1420,15-418,15-446,16-178,16-178_Normal,DKO,HCT116,SW480
0,1,890000,895000,insul_178,0.721850,0.901089,0.839639,0.330144,0.312873,0.727936,...,0.256476,1.384033,0.912626,0.379787,0.659389,0.668441,1.320379,0.998001,1.662335,0.989169
1,1,895000,900000,insul_179,0.540577,0.595470,0.650683,0.389669,-0.254923,0.668293,...,0.210331,0.905053,0.605062,0.557428,0.368272,0.516904,0.907322,0.891386,1.520709,0.562468
2,1,900000,905000,insul_180,0.418510,0.642061,0.561610,0.451869,-0.214719,0.916896,...,-0.009144,0.866584,0.759631,0.540593,0.420482,0.781214,0.543994,0.787745,1.590608,0.251199
3,1,905000,910000,insul_181,0.497176,0.356007,0.261403,0.214934,0.068536,0.341432,...,-0.015539,0.695639,0.733940,0.361112,0.640313,0.426678,-0.188751,0.682265,1.415113,0.359788
4,1,910000,915000,insul_182,0.814550,0.834990,0.563191,0.274900,0.074303,0.271685,...,0.370163,0.901901,0.828751,0.348402,0.670816,0.333805,-0.137275,0.779230,1.365818,0.674101


In [20]:
expr_new.to_csv('/cluster2/home/futing/Project/HiCQTL/merged/CRC53g/phenotype/CRC53g_index.bed',sep='\t')

In [24]:
df=expr.drop(columns=['chr','start','end','gene']).T
# df=expr.drop(columns=['gene_id']).T
# 1. 标准化数据（PCA 对量纲很敏感）
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df)

# 2. 建立 PCA 模型，保留前 2 个主成分
pca = PCA(n_components=20)
X_pca = pca.fit_transform(X_scaled)

In [25]:
# 整理列名
expr_pc = pd.DataFrame(X_pca)
expr_pc.columns = ["exprPC" + str(i) for i in range(1,21)]
expr_pc.index = df.index
# expr_pc = expr_pc.T
expr_pc.head()

,exprPC1,exprPC2,exprPC3,exprPC4,exprPC5,exprPC6,exprPC7,exprPC8,exprPC9,exprPC10,exprPC11,exprPC12,exprPC13,exprPC14,exprPC15,exprPC16,exprPC17,exprPC18,exprPC19,exprPC20
09-376,-31.976018,-8.173653,-56.728073,-24.824150,-42.609133,46.566065,-119.783207,114.719255,144.937114,-115.090056,-62.655660,86.250014,-23.737193,-136.722967,37.503773,-55.829045,-462.742842,-34.313060,-246.677730,82.439331
11-1026,20.582961,-3.679513,7.609231,29.120114,39.418954,59.904715,-26.260323,-0.666077,-33.025812,12.923755,-16.321169,13.254801,92.958471,-32.637979,-9.026751,-45.039703,-110.840669,-97.607696,-142.826512,81.020550
11-1074,27.778831,12.793910,37.569293,43.571628,30.710046,51.904070,3.542298,-16.913259,-26.539122,-1.415685,-18.362404,22.341523,132.391917,-18.299236,5.764554,-7.803836,-40.102647,-187.518402,-1.065699,-5.125930
11-1074_Normal,-227.566639,32.982577,-14.408790,-86.685637,-26.275521,-3.844950,2.903258,62.423914,7.733943,-30.297376,127.079595,9.318243,-37.157772,35.543128,11.172781,-47.964698,41.554029,95.694428,-114.313837,45.769599
11-51,13.088487,-1013.466701,234.911409,-82.324911,-43.933275,-54.503549,22.520579,-1.499334,-8.207240,4.097196,7.857828,4.890419,-8.932804,15.765587,-6.517888,3.979650,5.655733,8.140963,6.365667,-11.238104


In [ ]:
# 合并 genoPC exprPC cooler
covariate1=pd.merge(covariate,expr_pc,    left_index=True, 
    right_index=True,
    how='inner')

In [28]:
covariate2=covariate1.reindex(filelist['#IID'])
print(covariate2.index.equals(pd.Index(filelist['#IID'])))
covariate2.index.name = 'ID'

True


In [30]:
# covariate1=covariate1.T
covariate2.T.to_csv('/cluster2/home/futing/Project/HiCQTL/merged/CRC53g/covariate/covariate27.txt',sep='\t',index=True)

In [29]:
covariate2.head()

,element,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,...,exprPC11,exprPC12,exprPC13,exprPC14,exprPC15,exprPC16,exprPC17,exprPC18,exprPC19,exprPC20
ID,,,,,,,,,,,,,,,,,,,,,
09-376,105179165,-0.012284,0.036774,-0.010078,-0.006224,0.003777,-0.001598,0.005694,0.005845,0.019016,...,-62.655660,86.250014,-23.737193,-136.722967,37.503773,-55.829045,-462.742842,-34.313060,-246.677730,82.439331
11-1026,102757999,-0.014879,0.034419,-0.011229,0.028991,0.006685,0.000259,0.009118,-0.007509,0.014384,...,-16.321169,13.254801,92.958471,-32.637979,-9.026751,-45.039703,-110.840669,-97.607696,-142.826512,81.020550
11-1074,95497276,-0.011783,0.044870,-0.020293,-0.002919,-0.061093,0.242603,-0.622112,0.057411,0.024371,...,-18.362404,22.341523,132.391917,-18.299236,5.764554,-7.803836,-40.102647,-187.518402,-1.065699,-5.125930
11-1074_Normal,90275817,-0.011526,0.044422,-0.020083,-0.007265,-0.058645,0.236657,-0.601014,0.055795,0.024659,...,127.079595,9.318243,-37.157772,35.543128,11.172781,-47.964698,41.554029,95.694428,-114.313837,45.769599
11-51,61174254,-0.015352,0.041232,-0.019142,0.046098,0.025357,-0.084334,0.178513,-0.019549,0.379304,...,7.857828,4.890419,-8.932804,15.765587,-6.517888,3.979650,5.655733,8.140963,6.365667,-11.238104
